## Part1: K-means with 10 clusters

### Load data and import packages

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from pyproj import CRS, Transformer
from sklearn.preprocessing import StandardScaler
from category_encoders import BinaryEncoder
from sklearn.metrics import davies_bouldin_score
from sklearn.preprocessing import PowerTransformer
import geopandas as gpd

In [2]:
divar_train = pd.read_csv('../Divar Dataset/Divar.csv')

C:\Users\farzin\AppData\Local\Temp\ipykernel_23268\4260200410.py:1: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  divar_train = pd.read_csv('../Divar Dataset/Divar.csv')


### Part 1: `Kmeans with 10 clusters`<br>
First we will try `K-means` on  4 different feature sets namely: `feature_set_[1,2,3,4]` to find a better clustering feature set.<br>
To achieve the best set, we then evaluate our clustering model with the following `Evaluation Metrics`:<br>

* `davies_bouldin_score`
* `Inertia` (SSE - Sum of Squared Errors)

Finally, we choose the `best feature set` to plot on Iran's map and show the clustering data comprehensively

In [3]:
divar_train_cp = divar_train.copy()

In [4]:
# Feature selection for K-means
feature_set_1 = divar_train_cp[['city_slug', 'price_value', 'building_size', 'cat3_slug']]
feature_set_2 = divar_train_cp[['location_latitude', 'location_longitude', 'price_value']]
feature_set_3 = divar_train_cp[['location_latitude', 'location_longitude', 'price_value', 'building_size', 'construction_year']]
feature_set_4 = divar_train_cp[['location_latitude', 'location_longitude', 'price_value', 'rent_value', 'credit_value']]

In [5]:
# Preprocessing Feature Set 1
# Handling missing values
feature_set_1 = feature_set_1.dropna()

# Encoding Nominal features
# Frequency Encoding for city_slug
freq_encoding = feature_set_1['city_slug'].value_counts().to_dict()
feature_set_1['city_slug'] = feature_set_1['city_slug'].map(freq_encoding)

# Binary Encoding for cat3_slug
be = BinaryEncoder()
temp = be.fit_transform(feature_set_1[['cat3_slug']])
feature_set_1 = pd.concat([feature_set_1, temp], axis=1)
feature_set_1.drop(columns=['cat3_slug'], inplace=True)

# Feature Scaling
scaler_1 = StandardScaler()
pf = PowerTransformer(method='yeo-johnson')
feature_set_1['price_value'] = pf.fit_transform(feature_set_1[['price_value']])
feature_set_1_scaled = scaler_1.fit_transform(feature_set_1)
feature_set_1_scaled = pd.DataFrame(feature_set_1_scaled, columns=feature_set_1.columns)

In [6]:
# Preprocessing Feature Set 2
# Handling missing values
feature_set_2 = feature_set_2.dropna()

# Feature Scaling
pf = PowerTransformer(method='yeo-johnson')
feature_set_2['price_value'] = pf.fit_transform(feature_set_2[['price_value']])
scaler_2 = StandardScaler()
feature_set_2_scaled = scaler_2.fit_transform(feature_set_2)
feature_set_2_scaled = pd.DataFrame(feature_set_2_scaled, columns=feature_set_2.columns)

In [7]:
# Preprocessing Feature Set 3
# Handling missing values
feature_set_3 = feature_set_3.dropna()

# Digits conversion to english
def persian_to_english(sample_input: str):
    persian_digits = '۰۱۲۳۴۵۶۷۸۹'
    english_digits = '0123456789'
    trans_table = str.maketrans(persian_digits, english_digits)
    return sample_input.translate(trans_table)

feature_set_3['construction_year'] = feature_set_3.loc[:, 'construction_year'].apply(
    lambda x: persian_to_english(x) if isinstance(x, str) else x)

# change to only digits
feature_set_3['construction_year'] = feature_set_3['construction_year'].replace('قبل از 1370', '1370')
# change to numeric type
feature_set_3['construction_year'] = pd.to_numeric(feature_set_3['construction_year'], errors='coerce')
feature_set_3['age'] = 1404 - feature_set_3['construction_year']
feature_set_3.drop(columns=['construction_year'], inplace=True)

# Feature Scaling
pf = PowerTransformer(method='yeo-johnson')
feature_set_3['price_value'] = pf.fit_transform(feature_set_3[['price_value']])
feature_set_3['building_size'] = pf.fit_transform(feature_set_3[['building_size']])
scaler_3 = StandardScaler()
feature_set_3_scaled = scaler_3.fit_transform(feature_set_3)
feature_set_3_scaled = pd.DataFrame(feature_set_3_scaled, columns=feature_set_3.columns)

In [8]:
# Preprocessing feature_set_4
# fill price_value missing values with transformed_price comming form it's rent and credit
def rent_to_price(rent, credit):
    if rent < 0 or credit < 0:
        return 0
    credit_total = credit + (rent * 100) / 3
    transformed_price = credit_total * 6
    return max(0, transformed_price)

# Handling missing values
feature_set_4.loc[:, 'price_value'] = feature_set_4.apply(
    lambda row: rent_to_price(row['rent_value'], row['credit_value']) if
      (pd.isna(row['price_value']) & pd.notna(row['rent_value']) & pd.notna(row['credit_value'])) else row['price_value'],
       axis=1)

feature_set_4 = feature_set_4.drop(columns=['rent_value', 'credit_value'], axis=1)
feature_set_4 = feature_set_4.dropna()

# Feature Scaling
feature_set_4_cp = feature_set_4.copy()
pf = PowerTransformer(method='yeo-johnson')
feature_set_4_cp['price_value'] = pf.fit_transform(feature_set_4_cp[['price_value']])
scaler_4 = StandardScaler()
feature_set_4_scaled = scaler_4.fit_transform(feature_set_4_cp)
feature_set_4_scaled = pd.DataFrame(feature_set_4_scaled, columns=feature_set_4_cp.columns)

#### Evaluation
As we mentioned above, we use `kmean` with `n_cluster=10` for the feature sets. <br>
Then we evaluate the trained models with <b>2 different metrics</b>.

In [9]:
k_values = 10
features_dict = {'Feature Set 1': feature_set_1_scaled,
                 'Feature Set 2': feature_set_2_scaled,
                 'Feature Set 3': feature_set_3_scaled,
                 'feature_set_4': feature_set_4_scaled
                 }
results_dict = {}
for feature_name, feature_data in features_dict.items():
    kmeans = KMeans(n_clusters=k_values, random_state=42)
    cluster_labels = kmeans.fit_predict(feature_data)
    davies_bouldin = davies_bouldin_score(feature_data, cluster_labels)
    results_dict[feature_name] = {
        'davies_bouldin': davies_bouldin,
        'Inertia': kmeans.inertia_
    }
    
print(pd.DataFrame(results_dict).T)


               davies_bouldin        Inertia
Feature Set 1        0.808282  637690.187921
Feature Set 2        0.901254  224791.580539
Feature Set 3        1.156761  585068.830427
feature_set_4        0.784098  378977.759181


#### Best Feature set: `feature_set_4`<br>
As we can see from the results, `Davis Bouldin` is the lowest for this feature set. So we now, make it as out <b>best choice</b><br>
Now we want to do `k-means` with `k=10` and with `feature_set_4`. But before that, we firts remove the data point whcih are not within `Iran's borders`. We use `geopandas` for this reason.

In [10]:
del feature_set_1, feature_set_1_scaled, feature_set_2, feature_set_2_scaled
del feature_set_3, feature_set_3_scaled, temp, freq_encoding, cluster_labels
del feature_data, feature_name, features_dict, feature_set_4_cp, feature_set_4_scaled, kmeans

In [11]:
# We want to get only datapoints which are within Iran's borders
# Build GeoDataFrame in WGS84
gdf = gpd.GeoDataFrame(
    feature_set_4,
    geometry=gpd.points_from_xy(
        feature_set_4["location_longitude"],
        feature_set_4["location_latitude"]
    ),
    crs="EPSG:4326"
)

# Load Iran border (Natural Earth)
iran = gpd.read_file(
    "https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip"
)
iran = iran[iran["NAME"] == "Iran"].to_crs("EPSG:4326")

# Spatial filter
iran_geom = iran.geometry.iloc[0]
gdf = gdf[gdf.within(iran_geom)].copy()


d:\ProgramData\miniconda3\envs\ai\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


In [12]:
# Add Iran's Zones
gdf['utm_zone'] = (np.floor((gdf['location_longitude'] + 180) / 6) + 1).astype(int)
print("Zones are: ", gdf['utm_zone'].unique())

Zones are:  [39 40 38 41]


In [13]:
# Adding UTM cordinations to the data points
def apply_utm_conversion(group):
    zone = group.name
    wgs84_crs = CRS.from_epsg(4326)
    utm_crs = CRS.from_epsg(32600 + zone)
    transformer = Transformer.from_crs(wgs84_crs, utm_crs, always_xy=True)
    
    easting, northing = transformer.transform(
        group['location_longitude'].values,
        group['location_latitude'].values
    )
    
    group['utm_easting'] = easting
    group['utm_northing'] = northing
    return group

gdf = gdf.groupby(['utm_zone'], group_keys=False).apply(apply_utm_conversion, include_groups=False)
gdf.head()

,location_latitude,location_longitude,price_value,geometry,utm_easting,utm_northing
2,35.703865,51.373459,9.700000e+09,POINT (51.37346 35.70387),533784.424602,3.951168e+06
7,35.729832,51.505466,8.700000e+09,POINT (51.50547 35.72983),545711.558185,3.954101e+06
8,35.712364,50.794781,6.500000e+08,POINT (50.79478 35.71236),481437.130969,3.952066e+06
10,35.778664,51.757549,2.600000e+09,POINT (51.75755 35.77866),568467.028494,3.959664e+06
11,35.733952,51.380608,7.200000e+09,POINT (51.38061 35.73395),534418.187237,3.954507e+06


In [16]:
# Feature Scaling
pf = PowerTransformer(method='yeo-johnson')
gdf['price_value'] = pf.fit_transform(gdf[['price_value']])
scaler_5 = StandardScaler()
gdf_scaled = scaler_5.fit_transform(gdf[['location_latitude', 'location_longitude', 'price_value']])
gdf_scaled = pd.DataFrame(gdf_scaled, columns=['location_latitude', 'location_longitude', 'price_value'])

In [17]:
kmeans = KMeans(n_clusters=10, random_state=42)
cluster_labels = kmeans.fit_predict(gdf_scaled)
gdf['cluster_number'] = cluster_labels

centers_scaled = kmeans.cluster_centers_
centers_original = scaler_5.inverse_transform(centers_scaled)

# location_latitude, location_longitude
centers_lat = centers_original[:, 0]  
centers_lon = centers_original[:, 1] 
gdf['price_value'] = pf.inverse_transform(gdf[['price_value']])

In [ ]:
# Plot the Map
fig = px.scatter_map(
    gdf, 
    lat='location_latitude', 
    lon='location_longitude', 
    color='cluster_number',
    hover_data=['price_value', 'utm_easting', 'utm_northing'],
    map_style="open-street-map",
    zoom=10, 
    center={"lat": 32.4279, "lon": 53.6880},
    title='Geographical Scatter Plot of samples with K-means Clusters (10 Clusters)',
    height=600,
)

# Add Cluster Centers
fig.add_trace(go.Scattermap(
    lat=centers_lat,
    lon=centers_lon,
    mode='markers+text',
    marker=dict(size=20, color='red', symbol='star'),
    text=[f'Cluster {i}' for i in range(len(centers_lat))],
    textposition="top center",
    textfont=dict(size=12),
    name='Cluster Centers',
    hovertemplate='<b>Cluster %{text}</b><br>Lat: %{lat}<br>Lon: %{lon}<extra></extra>'
))

fig.show()

